# Assignment 7: Semantic Segmentation
Heute werden wir einfache Netzwerkarchitekturen für "Semantic Segmentation" testen. Ziel ist es dieses Paper in den Grundzügen zu implementieren: https://people.eecs.berkeley.edu/~jonlong/long_shelhamer_fcn.pdf. Bitte lesen!

## Daten

Es gibt einige gute Datensätze, die ihr (bei gegebener Hardware) herunterladen und benutzen könnt. Für diejenigen, die auf CPUs rechnen gilt immer der Tip: Bilder downsamplen!

Sucht Euch einen Satensatz aus: 

KITTI: http://www.cvlibs.net/download.php?file=data_semantics.zip (~300 MB, 200 Bilder)

DUS: http://www.6d-vision.com/scene-labeling (~3 GB, 500 Bilder)

MIT. http://sceneparsing.csail.mit.edu/ (~1 GB, links siehe auf Seite)

## Exc. 7.1 Fully convolutional network, no downsampling
Implementiere die in der Vorlesung besprochene Netzwerkarchitektur von aufeinanderfolgenden CONV-Schichten (stride=1, mit zero-padding), um eine Ausgabeschicht zu bekommen, die die Eingabegröße aufweist. Tip: die letzte CONV-Schicht sollte eine Tiefe haben, die zur Zahl der Klassen korrespondiert. Benutze den L2-Loss zum Labelbild (Achtung: ihr müsst dafür entweder das Labelbild oder den Ausgabetensor umformulieren).

Trainiere das Netzwerk auf den von Dir gewählten Datensatz und zeige den Verlauf des Losses, und einige zufällig gewählte Beispielbilder mit ihren vorhergesagten Segmentierungen an. (**RESULT**)

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import itertools
import tensorflow as tf

from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.preprocessing import binarize

from keras.backend.tensorflow_backend import set_session, get_session
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, decode_predictions
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.utils.np_utils import to_categorical
from keras import optimizers

from skimage import color, io

import warnings; warnings.simplefilter('ignore')

%matplotlib inline

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

def preprocess(img):
    return color.hsv2rgb(img)

def loadImageArray(path):
    list = []
    for f in os.listdir(path):
        ext = os.path.splitext(f)[1]
        if ext.lower() not in [".png"]:
            continue
        list.append(color.hsv2rgb(io.imread(os.path.join(path, f))))
    return np.copy(list)


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
image_generator = ImageDataGenerator()
label_generator = ImageDataGenerator()

# Beispiel für den KITTI-Datensatz. Ich habe die 200 training samples in 180 train- und 20 testbilder
# geteilt (macht 180 samples inkl. labels)
# um uns das Leben leichter zu machen, Bilder heruntersamplen
img_size = (38, 125)

train_image_path = os.path.join("semantic_segmentation", "train", "images")
train_label_path = os.path.join("semantic_segmentation", "train", "labels")

print(train_image_path)

train_images = loadImageArray(train_image_path + "/images")

print(len(train_images))

plt.figure(1, figsize=(20,7))
plt.imshow(train_images[0])
plt.show()

In [ ]:
input_shape = (img_size[0], img_size[1], 3)
lr = 0.001

model = Sequential()

model.add(Conv2D(32, img_size, activation="relu", padding="same", input_shape=input_shape))
model.add(Conv2D(16, img_size, activation="relu", padding="same"))
model.add(Conv2D(12, img_size, activation="relu", padding="same"))
model.add(Conv2D(12, img_size, activation="relu", padding="same"))
model.add(Conv2D(6, img_size, activation="relu", padding="same"))
model.add(Conv2D(3, img_size, activation="relu", padding="same"))

model.compile(loss='mean_squared_error',
              optimizer=optimizers.Adam(lr=lr),
              metrics=['accuracy'])

model.summary()

In [ ]:
# nach der Definition des Modells:
# model.fit_generator(train_generator, steps_per_epoch = 1000, epochs = 10)
model.fit_generator(train_generator, steps_per_epoch = 50, epochs = 5)


In [1]:
model.save('models/ss-4406.h5')
model.save_weights('models/ss-weights-4406.h5')
#model.load_weights('models/ss-weights-4406.h5')


NameError: name 'model' is not defined

In [ ]:
test_image_path = os.path.join("semantic_segmentation", "test", "images")
test_label_path = os.path.join("semantic_segmentation", "test", "labels")

testImg = image_generator.flow_from_directory(test_image_path,
                                        class_mode=None,
                                        target_size=img_size, 
                                        seed=1)

testLbl = image_generator.flow_from_directory(test_label_path,
                                        class_mode=None,
                                        target_size=img_size, 
                                        seed=1)

test = map(testImg, testLbl)

In [ ]:
# print(testImg.next()[0].shape)
image = testImg.next()[0,:,:,:]
predictions = model.predict_generator(testImg)

In [ ]:
plt.figure(1, figsize=(20,7))
img = preprocess(image.astype(int))
plt.imshow(img[:,:,2])
plt.show()

In [ ]:
print(predictions.shape)
plt.figure(1, figsize=(20,7))
plt.imshow(predictions[1,:,:,0].astype(int))
plt.show()

# Exc. 7.2 FCN mit Bottleneck

Implementiere jetzt die Variante mit schrittweisem Down- und Upsampling, wie in der Vorlesung besprochen. Nutze dafür ein bestehendes Netzwerk (z.B. VGG16, https://keras.io/applications/#vgg16), entferne die FC-Schichten am Ende, und füge dann die Upsampling-Schichten hinzu. Wie in der vorigen Vorlesung zu Transfer Learning beschrieben, kannst Du jetzt nur den zweiten Teil trainieren und die Gewichte des ersten Teils "einfrieren".

Stelle wie oben den Verlauf des Losses dar und wähle einige Beispielbilder aus dem Testset und zeige sie mit ihrer vorhergesagten Segmentierung an. (**BONUS**)